In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor,as_completed

C:\Users\鲁鲁\AppData\Local\Temp\ipykernel_6688\268587628.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
urls_path="../Result/cleaned_url.csv"
urls=pd.read_csv(urls_path)
urls=list(urls['0'])

In [3]:
urls

['https://dblp.uni-trier.de/pid/66/3027.html',
 'https://dblp.org/pid/06/9379.html',
 'https://dblp.org/pid/07/1181.html',
 'https://dblp.org/pid/m/RJMiller.html',
 'https://dblp.uni-trier.de/pid/84/2154.html',
 'https://dblp.uni-trier.de/pid/97/3036.html',
 'https://dblp.org/pid/18/2478.html',
 'https://dblp.org/pid/80/2320.html',
 'https://dblp.org/pid/m/SamuelMadden.html',
 'https://dblp.org/pid/127/0138.html',
 'https://dblp.org/pid/55/3364.html',
 'https://dblp.org/pid/135/4661.html',
 'https://dblp.org/pid/09/3737.html',
 'https://dblp.uni-trier.de/pid/204/3779.html',
 'https://dblp.org/pid/k/BettinaKemme.html',
 'https://dblp.org/pid/127/6172-1.html',
 'https://dblp.org/pid/01/4316-1.html',
 'https://dblp.org/pid/128/2991.html',
 'https://dblp.org/pid/97/9726.html',
 'https://dblp.org/pid/75/887.html',
 'https://dblp.org/pid/d/AmolDeshpande.html',
 'https://dblp.org/pid/143/0258-1.html',
 'https://dblp.org/pid/68/6119.html',
 'https://dblp.org/pid/83/4809-2.html',
 'https://dblp

In [4]:
def fetch(url):
    response=requests.get(url)
    if response.status_code==200:
        return response.text
    elif response.status_code==429:
        wait_time=int(response.headers['Retry-After']+10)
        print(f"Waiting {wait_time} seconds")
        time.sleep(wait_time)
        return fetch(url)

In [5]:
def parse(url):
    soup=BeautifulSoup(fetch(url),'html.parser')
    return soup

In [6]:
#eu=urls[0]

In [7]:
#eu

In [8]:
#soup=parse(eu)

In [9]:
def get_name(soup):
    name=soup.find_all('span',"name primary")[0].getText()
    return name

In [10]:
#get_name(soup)

In [11]:
#publ=soup.find('div',id="publ-section")
#publ=publ.find_all('ul',class_="publ-list")

In [12]:
#publ

In [13]:
def get_publish_info(li):
    info={}
    #i1=li.find('cite',class_="data tts-content")
    #print(i1)
    #print(li)
    ico=li.find_all('span',itemprop="author")
    coo_list=[]
    for co in ico:
       coo=co.find('a')
       if coo:
           if coo['href'] in urls:
            coo_list.append(coo['href'])
    info['co_authors']=coo_list
    title=li.find('span',class_="title").getText()
    #print(title)
    info['title']=title
    
    return info

In [14]:
def get_publish_info1(pu):
    publish={}
    for li in pu.find_all('li'):
    #print(li)
        if 'class' in li.attrs and 'year' in li.attrs['class']:
            current_year=li.getText()
            
            publish[current_year]=[]
        elif 'class' in li.attrs and 'entry' in li.attrs['class']:
            info=get_publish_info(li)
            info['year']=current_year
            publish[current_year].append(info)
    return publish
        #print(info)

In [15]:
#publish

In [16]:
def merge_coo_in_year(dictlist):
    coo_list=[]
    for d in dictlist:
        coo_list.extend(d['co_authors'])
    coo_list=list(set(coo_list))
    return coo_list

In [17]:
# year_coo={}
# for year in publish:
#     year_coo[year]=merge_coo_in_year(publish[year])

In [18]:
# year_coo

In [19]:
def get_coo_info_by_year_from_url(url):
    global count
    soup=parse(url)
    name=get_name(soup)
    publ=soup.find('div',id="publ-section")
    publ=publ.find_all('ul',class_="publ-list")
    # publish={}
    # for li in publ[0].find_all('li'):
    #     #print(li)
    #     if 'class' in li.attrs and 'year' in li.attrs['class']:
    #         current_year=li.getText()
    # 
    #         publish[current_year]=[]
    #     elif 'class' in li.attrs and 'entry' in li.attrs['class']:
    #         info=get_publish_info(li)
    #         info['year']=current_year
    # 
    #         publish[current_year].append(info)
    #         # print(info)
    pub={}
    for pu in publ:
        publish=get_publish_info1(pu)
        pub.update(publish)
        pub.update(publish)
    
    year_coo={}
    for year in pub:
        year_coo[year]=merge_coo_in_year(pub[year])

    count+=1
    print(count,name)
    return name,year_coo

In [20]:
#get_coo_info_by_year_from_url(eu)

In [21]:
coo_relation={}
name_url={}
count=0
with ThreadPoolExecutor(max_workers=15) as executor:
    f_to_url={executor.submit(get_coo_info_by_year_from_url,url):url for url in urls}
    for future in as_completed(f_to_url):
        url=f_to_url[future]
        try:
            n,data=future.result()
            coo_relation[url]=data
            name_url[url]=n
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))

1 Yuval Moskovitch
2 Suppanut Pothirattanachaikul
3 Yadollah Yaghoobzadeh
4 Barna Saha
5 Jaroslaw Szlichta
6 Lipika Dey
7 Pratibha Moogi
8 Mohammad Dashti 0001
9 Eric Lo 0001
10 Kaiqi Zhao 0001
11 Renée J. Miller
12 Andreas Züfle
13 Xiaofei Zhang 0002
14 Elena Demidova
15 Bettina Kemme
16 Zengfeng Huang
17 Matthias Boehm 0001
18 Renqin Cai
19 Panos Kalnis
20 Yuchen Li 0001
21 Kok-Leong Ong
22 Weining Qian
23 Orestis Polychroniou
24 Ming Li 0006
25 Vineet Chaoji
26 Gautam Das 0001
27 Hainan Zhang
28 Niyati Chhaya
29 Linmei Hu
30 Choiru Za'in
31 Amol Deshpande
32 Nikolay Yakovets
33 Narayanan Unny Edakunni
34 Tamer Elsayed
35 AnHai Doan
3637 Mayank Singh 0001
 Iraklis Psaroudakis
38 Lyublena Antova
39 Aline Bessa
40 Yongbin Liu
41 Ming Liu 0018
42 Monica Scannapieco
43 Samuel Madden 0001
44 Saket Sathe 0001
45 Felix Naumann
46 Konstantinos Karanasos
47 Yunyao Li 0001
48 Tony Tan
49 Javier Parapar
50 Vassilis Christophides
51 Evaggelia Pitoura
52 Lina Yao
53 Immanuel Trummer
54 Yingyi Bu


In [22]:
len(coo_relation)

1058

In [23]:
#save as json
import json
with open('../Result/coo_relation.json', 'w') as fp:
    json.dump(coo_relation, fp)

In [24]:
with open('../Result/name_url.json', 'w') as fp:
    json.dump(name_url, fp)

In [25]:
url_index={}
for i in range(len(urls)):
    url_index[urls[i]]=i

In [26]:
with open('../Result/url_index.json', 'w') as fp:
    json.dump(url_index, fp)